In [95]:
import folium
from folium.features import GeoJsonTooltip
import fiona
import geopandas
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import pandas as pd
from folium.plugins import HeatMap

import pandas as pd
import geopandas as gpd

import geopandas.tools
#from geopy.geocoders import Nominatim
#import geopy
from geopandas.tools import geocode
#from geopy.extra.rate_limiter import RateLimiter
from difflib import SequenceMatcher

from folium.plugins import HeatMap, MarkerCluster, HeatMapWithTime

import numpy as np

from shapely.geometry import Point, Polygon
from collections import defaultdict, OrderedDict

<h3> Create HeatMapWithTime for start stations 2011 </h3>

Going from January to December, month by month

In [96]:
stations_2011 = pd.read_csv('cleaned_data/stations_2011.csv')

In [97]:
df_2011 = pd.read_csv('cleaned_data/df_2011.csv')

In [98]:
df_start_2011 = df_2011[['start_date', 'start_station', 'registered', 'casual']]

In [99]:
# To create a heat map using HeatMapWithTime:
# "data" parameter: a list of lists; each nested list contains [lat, lon, number]. The number defines the density on the heatmap. 
# "index" parameter: a list of indexes; each index corresponds to the time unit (in this case, "month", from 0 to 12)

df_start_2011['start_date']= pd.to_datetime(df_start_2011['start_date']).dt.to_period('M')
dummy = df_start_2011.groupby(['start_date', 'start_station']).sum().reset_index()
dummy = dummy.merge(stations_2011, left_on = 'start_station', right_on = 'station').drop(['station', 'region'], axis = 1)
dummy['month'] = dummy['start_date'].dt.month


# stations11_heat_start_reg is a dictionary. 
# The keys are the months (0 to 12), the values are lists of lists. 

stations11_heat_start_reg = defaultdict(list)       # Create defaultdict, whose values are lists
for row in dummy.itertuples():
    stations11_heat_start_reg[row.month].append([row.lat, row.lon, row.registered]) # In each list (value), insert a list [lat, lon, number of registered users]

# make sure the keys are in the right order
stations11_heat_start_reg = OrderedDict(sorted(stations11_heat_start_reg.items()))

/var/folders/68/lmdx_4hd4msbstk8y2t5phyh0000gn/T/ipykernel_78808/2957064798.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_start_2011['start_date']= pd.to_datetime(df_start_2011['start_date']).dt.to_period('M')


In [100]:
m=folium.Map(location=[38.900497,-77.007507],zoom_start=12,min_zoom=11,zoom_control=False,Control_scale=True)
hm = HeatMapWithTime(data=list(stations11_heat_start_reg.values()),
                     index=list(stations11_heat_start_reg.keys()), 
                     radius=10,
                     auto_play=False,
                     max_opacity=0.3)
hm.add_to(m)
m

<h3> Now we want to create a heat map with time, going year by year, from 2011 to 2017 </h3>


In [160]:
stations_2011 = pd.read_csv('cleaned_data/stations_2011.csv')
stations_2012 = pd.read_csv('cleaned_data/stations_2012.csv')
stations_2013 = pd.read_csv('cleaned_data/stations_2013.csv')
stations_2014 = pd.read_csv('cleaned_data/stations_2014.csv')
stations_2015 = pd.read_csv('cleaned_data/stations_2015.csv')
stations_2016 = pd.read_csv('cleaned_data/stations_2016.csv')
stations_2017 = pd.read_csv('cleaned_data/stations_2017.csv')
df_2011 = pd.read_csv('cleaned_data/df_2011.csv')
df_2012 = pd.read_csv('cleaned_data/df_2012.csv')
df_2013 = pd.read_csv('cleaned_data/df_2013.csv')
df_2014 = pd.read_csv('cleaned_data/df_2014.csv')
df_2015 = pd.read_csv('cleaned_data/df_2015.csv')
df_2016 = pd.read_csv('cleaned_data/df_2016.csv')
df_2017 = pd.read_csv('cleaned_data/df_2017.csv')

In [176]:
stations_all_years = [stations_2011, stations_2012, stations_2013, stations_2014, stations_2015, stations_2016, stations_2017]
df_all_years = [df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017]

In [171]:
df_2011['start_date'] = pd.to_datetime(df_2011['start_date'])

In [177]:
# Convert "start_date" column in df_20xx into datetime format
for df_year in df_all_years:
    df_year['start_date'] = pd.to_datetime(df_year['start_date'])

In [ ]:
df_start_all_years = []

for i in range(len(df_all_years)):
    df_temp = df_all_years[i][['start_date', 'start_station', 'registered', 'casual']]
    df_temp = df_temp.groupby(['start_date', 'start_station']).sum().reset_index()
    df_temp['year'] = df_temp['start_date'].dt.year
    df_temp = df_temp.groupby(['year', 'start_station']).sum().reset_index()
    df_temp = df_temp.merge(stations_all_years[i], left_on = 'start_station', right_on = 'station').drop(['station', 'region'], axis = 1)
    df_start_all_years.append(df_temp)

df_start_2011 = df_start_all_years[0]
df_start_2012 = df_start_all_years[1]
df_start_2013 = df_start_all_years[2]
df_start_2014 = df_start_all_years[3]
df_start_2015 = df_start_all_years[4]
df_start_2016 = df_start_all_years[5]
df_start_2017 = df_start_all_years[6]


In [187]:
df_start_combined = pd.concat([df_start_2011, df_start_2012, df_start_2013, df_start_2014, df_start_2015, df_start_2016, df_start_2017])

In [199]:
from sklearn.preprocessing import minmax_scale

In [200]:
df_start_combined['registered'] = minmax_scale(df_start_combined['registered'])
df_start_combined['casual'] = minmax_scale(df_start_combined['casual'])

In [201]:
df_start_combined [df_start_combined['year']==2011].head()

,year,start_station,registered,casual,lat,lon
0,2011,10th & Monroe St NE,0.031697,0.005821,38.932457,-76.993534
1,2011,10th & U St NW,0.236608,0.032184,38.917200,-77.025900
2,2011,10th St & Constitution Ave NW,0.123232,0.153641,38.893028,-77.026013
3,2011,11th & F St NW,0.000349,0.000000,38.897857,-77.026975
4,2011,11th & H St NE,0.003286,0.000123,38.900164,-77.005441


In [202]:
df_start_combined [df_start_combined['year']==2017].head()

,year,start_station,registered,casual,lat,lon
0,2017,10th & E St NW,0.132644,0.124042,38.895914,-77.026064
1,2017,10th & Florida Ave NW,0.129454,0.014668,38.920387,-77.025672
2,2017,10th & G St NW,0.126280,0.065230,38.898243,-77.026235
3,2017,10th & K St NW,0.230687,0.085244,38.902400,-77.026220
4,2017,10th & Monroe St NE,0.047585,0.010605,38.932457,-76.993534


In [190]:
stations_start_reg = defaultdict(list)
for row in df_start_combined.itertuples():
    stations_start_reg[row.year].append([row.lat, row.lon, row.registered])
stations_start_reg = OrderedDict(sorted(stations_start_reg.items()))

In [207]:
m=folium.Map(location=[38.900497,-77.007507],zoom_start=12,min_zoom=11,zoom_control=False,Control_scale=True)
hm = HeatMapWithTime(data=list(stations_start_reg.values()),
                     index=list(stations_start_reg.keys()), 
                     radius=20,
                     auto_play=False,
                     max_opacity=0.5)
hm.add_to(m)
m

In [ ]:
df_2011_start = df_2011[['start_date', 'start_station', 'registered', 'casual']]
df_2011_start = df_2011_start.groupby(['start_date', 'start_station']).sum().reset_index()
df_2011_start['year'] = df_2011_start['start_date'].dt.year
df_2011_start = df_2011_start.groupby(['year', 'start_station']).sum().reset_index()
df_2011_start = df_2011_start.merge(stations_2011, left_on = 'start_station', right_on = 'station').drop(['station', 'region'], axis = 1)

In [158]:
df_2011_start.head()

,year,start_station,registered,casual,lat,lon
0,2011,10th & Monroe St NE,1997,331,38.932457,-76.993534
1,2011,10th & U St NW,14907,1830,38.917200,-77.025900
2,2011,10th St & Constitution Ave NW,7764,8736,38.893028,-77.026013
3,2011,11th & F St NW,22,0,38.897857,-77.026975
4,2011,11th & H St NE,207,7,38.900164,-77.005441
